<a href="https://colab.research.google.com/github/DA04/machine_learning_course/blob/main/MovieLens_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Использовать датасет MovieLens.
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
- TF-IDF на тегах и жанрах;
- средние оценки (+ median, variance и т. д.) пользователя и фильма
3. Оценить RMSE на тестовой выборке

In [177]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [178]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-05-04 10:55:42--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.70.113, 74.125.70.102, 74.125.70.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.70.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fu7bqgntv8h19c7d0toejudrpoi9i1ua/1683197700000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=6df3bfef-3453-46ac-8b8c-9766130e3b7a [following]
--2023-05-04 10:55:43--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fu7bqgntv8h19c7d0toejudrpoi9i1ua/1683197700000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=6df3bfef-3453-46ac-8b8c-9766130e3b7a
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 209.85.146.132, 2607:f8b0:4001:c1f::84
Connecting to doc-0g-84-

In [179]:
!unzip MovieLens.zip


Archive:  MovieLens.zip
replace links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [180]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [181]:
movies.head()
movies.value_counts()

movieId  title                                                  genres                                     
1        Toy Story (1995)                                       Adventure|Animation|Children|Comedy|Fantasy    1
53322    Ocean's Thirteen (2007)                                Crime|Thriller                                 1
53129    Mr. Brooks (2007)                                      Crime|Drama|Thriller                           1
53138    Librarian: Return to King Solomon's Mines, The (2006)  Action|Adventure|Fantasy                       1
53140    Librarian: Quest for the Spear, The (2004)             Action|Adventure|Comedy|Fantasy|Romance        1
                                                                                                              ..
4390     Rape Me (Baise-moi) (2000)                             Crime|Drama|Thriller                           1
4392     Alice (1990)                                           Comedy|Drama|Fantasy|Romance         

In [182]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [183]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [184]:
movie_ratings = ratings[['movieId', 'rating']].groupby(['movieId']).agg(["mean", "median", "var"]).reset_index()
movie_ratings.droplevel(level=0, axis=1).set_axis(['movieId', 'movie_rating_mean', 'movie_rating_median', 'movie_rating_var'], axis=1, copy=False).head()

,movieId,movie_rating_mean,movie_rating_median,movie_rating_var
0,1,3.920930,4.0,0.696990
1,2,3.431818,3.5,0.777419
2,3,3.259615,3.0,1.112651
3,4,2.357143,3.0,0.726190
4,5,3.071429,3.0,0.822917


In [185]:
user_ratings = ratings[['userId', 'rating']].groupby(['userId']).agg(["mean", "median", "var"]).reset_index()
user_ratings_df = user_ratings.droplevel(level=0, axis=1).set_axis(['userId', 'user_rating_mean', 'user_rating_median', 'user_rating_var'], axis=1, copy=False)
user_ratings_df.head()

,userId,user_rating_mean,user_rating_median,user_rating_var
0,1,4.366379,5.0,0.640077
1,2,3.948276,4.0,0.649015
2,3,2.435897,0.5,4.370783
3,4,3.555556,4.0,1.727132
4,5,3.636364,4.0,0.980973


In [186]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [188]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [189]:
movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[:5]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy']

In [190]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)

In [191]:
genres_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
genres_df.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [192]:
movies_df = pd.concat([movies, genres_df], axis = 1)
movies_df

,movieId,title,genres,action,adventure,animation,children,comedy,crime,documentary,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,193585,Flint (2017),Drama,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [193]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [194]:
tags['tag_lower'] = tags['tag'].str.lower()
tags['freq_count'] = tags.groupby('tag_lower')['tag_lower'].transform('count')
tags_freq = tags[tags['freq_count'] > 5].copy()
tags_freq['tags'] = tags_freq[['userId','movieId','tag_lower']].groupby(['userId','movieId'])['tag_lower'].transform(lambda x: '|'.join(x))
tags_df = tags_freq[['userId','movieId','tags']].drop_duplicates()
tags_df = tags_df.reset_index()

In [195]:
tags_df

,index,userId,movieId,tags
0,0,2,60756,funny|will ferrell
1,6,2,106782,drugs|leonardo dicaprio
2,12,18,431,mafia
3,14,18,1221,mafia
4,15,18,5995,holocaust
...,...,...,...,...
1045,3593,599,1732,black comedy|classic|coen brothers|comedy|crim...
1046,3624,599,2959,action|atmospheric|classic|clever|crime|dark|d...
1047,3672,600,273,gothic
1048,3673,606,1357,music


In [196]:
tags_df.count()

index      1050
userId     1050
movieId    1050
tags       1050
dtype: int64

In [197]:
movie_tags = [change_string(g) for g in tags_df.tags.values]
movie_tags[:5]

['funny willferrell', 'drugs leonardodicaprio', 'mafia', 'mafia', 'holocaust']

In [198]:
tfidf = TfidfVectorizer()
X_tag_train_tfidf = tfidf.fit_transform(movie_tags)
tag_df = pd.DataFrame(X_tag_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
tag_df.head()

,action,adolescence,adultery,adventure,aliens,animalmovie,animation,anime,assassination,astaireandrogers,...,twistending,vietnam,violence,visuallyappealing,visuallystunning,wedding,willferrell,witty,worldwarii,zombies
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.766472,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [199]:
tag_df.count()

action         1050
adolescence    1050
adultery       1050
adventure      1050
aliens         1050
               ... 
wedding        1050
willferrell    1050
witty          1050
worldwarii     1050
zombies        1050
Length: 135, dtype: int64

In [200]:
movies_tags_df = pd.concat([tags_df, tag_df], axis = 1)
movies_tags_df = movies_tags_df.drop(columns = ["index"])
movies_tags_df

,userId,movieId,tags,action,adolescence,adultery,adventure,aliens,animalmovie,animation,...,twistending,vietnam,violence,visuallyappealing,visuallystunning,wedding,willferrell,witty,worldwarii,zombies
0,2,60756,funny|will ferrell,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.766472,0.0,0.0,0.0
1,2,106782,drugs|leonardo dicaprio,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,18,431,mafia,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,18,1221,mafia,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,18,5995,holocaust,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,599,1732,black comedy|classic|coen brothers|comedy|crim...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.285272,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1046,599,2959,action|atmospheric|classic|clever|crime|dark|d...,0.202167,0.0,0.0,0.0,0.0,0.0,0.0,...,0.19383,0.0,0.227258,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1047,600,273,gothic,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1048,606,1357,music,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [201]:
df_1 = movies_with_ratings.merge(user_ratings_df, on = "userId", how="left")
df_1 = df_1.drop(columns = ["genres","timestamp","title"])
df_1.head()

,movieId,userId,rating,user_rating_mean,user_rating_median,user_rating_var
0,1,1,4.0,4.366379,5.0,0.640077
1,1,5,4.0,3.636364,4.0,0.980973
2,1,7,4.5,3.230263,3.5,1.767820
3,1,15,2.5,3.448148,3.5,1.284605
4,1,17,4.5,4.209524,4.0,0.258562


In [202]:
df_2 = df_1.merge(movies_df, on = "movieId", how="left")
df_2 = df_2.drop(columns = ["genres","title"])
df_2.head()

,movieId,userId,rating,user_rating_mean,user_rating_median,user_rating_var,action,adventure,animation,children,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,1,4.0,4.366379,5.0,0.640077,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5,4.0,3.636364,4.0,0.980973,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,7,4.5,3.230263,3.5,1.767820,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,15,2.5,3.448148,3.5,1.284605,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,17,4.5,4.209524,4.0,0.258562,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [203]:
df_3 = df_2.merge(movies_tags_df, on = ["movieId","userId"], how="left")
df_3 = df_3.drop(columns = ["tags"])
df_3 = df_3.fillna(0)
df_3.head()

,movieId,userId,rating,user_rating_mean,user_rating_median,user_rating_var,action_x,adventure_x,animation_x,children,...,twistending,vietnam,violence,visuallyappealing,visuallystunning,wedding,willferrell,witty,worldwarii,zombies
0,1,1,4.0,4.366379,5.0,0.640077,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5,4.0,3.636364,4.0,0.980973,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,7,4.5,3.230263,3.5,1.767820,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,15,2.5,3.448148,3.5,1.284605,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,17,4.5,4.209524,4.0,0.258562,0.0,0.416846,0.516225,0.504845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [204]:
X = df_3.drop(columns = ["movieId","userId","rating"])
y = pd.Series(data=df_3['rating'])
X

,user_rating_mean,user_rating_median,user_rating_var,action_x,adventure_x,animation_x,children,comedy_x,crime_x,documentary,...,twistending,vietnam,violence,visuallyappealing,visuallystunning,wedding,willferrell,witty,worldwarii,zombies
0,4.366379,5.0,0.640077,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.636364,4.0,0.980973,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.230263,3.5,1.767820,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.448148,3.5,1.284605,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.209524,4.0,0.258562,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,3.705224,4.0,1.139897,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100832,3.705224,4.0,1.139897,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100833,3.705224,4.0,1.139897,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100834,3.705224,4.0,1.139897,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [205]:
with pd.option_context('display.max_rows', None):
    print(df_3.dtypes)

movieId                 int64
userId                  int64
rating                float64
user_rating_mean      float64
user_rating_median    float64
user_rating_var       float64
action_x              float64
adventure_x           float64
animation_x           float64
children              float64
comedy_x              float64
crime_x               float64
documentary           float64
drama                 float64
fantasy_x             float64
filmnoir              float64
horror                float64
imax                  float64
musical               float64
mystery_x             float64
nogenreslisted        float64
romance_x             float64
scifi_x               float64
thriller              float64
war                   float64
western               float64
action_y              float64
adolescence           float64
adultery              float64
adventure_y           float64
aliens                float64
animalmovie           float64
animation_y           float64
anime     

In [211]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
model = linear_model.LinearRegression()

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [213]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [217]:
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred))

RMSE: 0.85


In [216]:
print ('actual:    ', y_test.values[0:5])
print ('predicted: ', y_pred[0:5])

actual:     [3.  4.5 4.  3.  5. ]
predicted:  [2.56615005 4.32198177 4.10906739 3.42570746 3.18514583]
